In [42]:
import requests
import pandas as pd
import sys
import os
# Добавляем путь к вашему файлу utils.py в sys.path
sys.path.append(r'C:\Users\123\Desktop\cards_flow_git')
from my_utils import load_api_tokens
import time

In [88]:
def cards_list_wb(api_token):
  """Получение информации о карточке товара"""
  url = 'https://content-api.wildberries.ru/content/v2/get/cards/list'
  headers = {
      "Authorization" : api_token
  }
  nmID = None
  updatedAt = None
  counter = 0
  total = 100
  limit = 100
  all_cards_data = []

  while total >= limit:
    print(f'Получены данные по {counter} карточкам')
    payload = {
      "settings": {
        "sort": {
          "ascending": False
        },
        "filter": {
          "withPhoto": -1
        },
        "cursor": {
          "updatedAt": updatedAt,
          "nmID": nmID,
          "limit": limit
        }
      }
    }
    try:
      res = requests.post(url, headers=headers, json=payload)
      print(res.status_code)
      data = res.json()
      if res.status_code == 200:
        nmID = data['cursor']['nmID']
        updatedAt = data['cursor']['updatedAt']
        total = data['cursor']['total']
        all_cards_data.append(data)
        counter+=total
    except Exception as e:
      print(e)
    time.sleep(1)
  return all_cards_data

cards_list = cards_list_wb(load_api_tokens()['Даниелян'])

Получены данные по 0 карточкам
200
Получены данные по 100 карточкам
200
Получены данные по 200 карточкам
200
Получены данные по 300 карточкам
200
Получены данные по 400 карточкам
200
Получены данные по 500 карточкам
200
Получены данные по 600 карточкам
200
Получены данные по 700 карточкам
200
Получены данные по 800 карточкам
200
Получены данные по 900 карточкам
200
Получены данные по 1000 карточкам
200
Получены данные по 1100 карточкам
200
Получены данные по 1200 карточкам
200
Получены данные по 1300 карточкам
200


In [89]:
cards_info = []
for cards in cards_list:
    for card in cards['cards']:
        cards_info.append(card)
len(cards_info)

1343

In [113]:
card_for_create = cards_info[1]

In [116]:
card_for_create

{'nmID': 220209029,
 'imtID': 241940459,
 'nmUUID': '018e8002-5d33-775e-bd45-71abdfca217c',
 'subjectID': 639,
 'subjectName': 'Увлажнители',
 'vendorCode': 'wild1196',
 'brand': 'Deerma',
 'title': 'Увлажнитель воздуха мини',
 'description': 'Увлажнитель воздуха для дома необходимая вещь, если у вас сухой воздух или начался зимний отопительный период. Deerma, являясь суббрендом Xiaomi выпустил качественный аромадиффузор для арома терапии эфирными маслами, которые позволят успокоится и насладиться приятными нотками. За счет нано-распыления поверхность стола остается чистой и не подвержена разложению. Очищенный воздух благодаря ультразвуковой пластины благоприятно повлияет на волосы, ногти и кожу в целом. Такая бытовая техника подойдет не только для квартиры, но и для офиса. Воздухоувлажнитель в комплекте дополнен проводом USB, что даст возможность использовать устройство от компьютера. Отметим, что его преимуществом также является подсветка с 8 лед режимами работы радужного перелива, к

In [114]:
subjectID = card_for_create['subjectID']
vendorCode = card_for_create['vendorCode']
title = card_for_create['title']
description = card_for_create['description']
brand = card_for_create['description']
try:
    card_for_create['dimensions'].pop('isValid')
    dimensions = card_for_create['dimensions']
except:
    print(f'Параметр isValid уже удален из словаря dimensions')
characteristics = card_for_create['characteristics']
sizes = card_for_create['sizes']

In [ ]:
def create_new_card(subjectID, vendorCode, title, description, brand, dimensions, characteristics, sizes):
    # Создание карточки в другом ЛК
    url = 'https://content-api.wildberries.ru/content/v2/cards/upload'
    payload = [
    {
        "subjectID": subjectID,
        "variants": [
        {
            "vendorCode": vendorCode,
            "title": title,
            "description": description,
            "brand": brand,
            "dimensions":dimensions,
            "characteristics": characteristics,
            "sizes": sizes
        }
        ]
    }
    ]
    headers = {
        "Authorization" : load_api_tokens()['Старт2']
    }

    errors_list = []
    try:
        req = requests.post(url, headers=headers, json=payload)
        print(f"Status Code: {req.status_code}")

        # Попробуем распарсить JSON только если Content-Type позволяет
        try:
            data = req.json()
        except requests.exceptions.JSONDecodeError:
            data = {"error": "Invalid JSON response", "raw": req.text}

        if req.status_code == 200:
            print(f"Создана карточка {title}")
            # Работа с data
        elif req.status_code == 400:
            error_text = data.get("errorText", "")
            print("Ошибка 400: Неверный запрос")
            print("Детали:", data)
            # Обрабатываю сценарии, когда карточка не создается:
            if "vendor code is used in other cards" in error_text:
                print("Ошибка в Wildberries: такой vendor code уже используется в другой карточке")
                errors_list.append(payload)  # сохраняем payload для анализа
            else:
                print(f"Неизвестная ошибка 400: {error_text}")
        elif req.status_code == 401:
            print("Ошибка 401: Неавторизован")
        elif req.status_code == 403:
            print("Ошибка 403: Доступ запрещён")
        elif req.status_code == 404:
            print("Ошибка 404: Ресурс не найден")
        elif req.status_code >= 500:
            print(f"Ошибка сервера {req.status_code}")
        else:
            print(f"Неожиданный статус: {req.status_code}, ответ: {data}")

    except requests.exceptions.RequestException as e:
        # Ошибка выполнения запроса
        print(f"Ошибка запроса: {e}")

Status Code: 200
Создана карточка Увлажнитель воздуха мини
